This notebook is to explore and implement Latent Dirichlet Allocation. <hr>

> Things to consider doing to improve results: plot distribution of words/lemmas in corpus and remove head and tail of distribution.

In [1]:
!pip install termite

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for termite: filename=termite-0.0.1-py3-none-any.whl size=6718 sha256=d3c6c61e9493b2f8e170bdb0820c3c1a76c3be97ab803af85fa5d9d377590619
  Stored in directory: /home/faten/.cache/pip/wheels/0e/e1/d4/ef7923a331dabfb3478d58567c7a377e02270488e169085202
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=6ef5dfa67b34e1bdd20dd786837f269570200a14d0d671f7f3ed54d2b15bdc22
  Stored in directory: /home/faten/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
  Created wheel for glob2: filename=glob2-0.7-py2.py3-none-any.whl size=9301 sha256=ae7f7c5d2b0a4150b29b9ce7bd349df4fbc03f75a156571817635c1da0a7b6a6
  Stored in directory: /home/faten/.cache/pip/wheels/96/41/06/9f8fddc6eb1d75bde63db7f491311a4ae26905212617e06eb2
Successfully built termite docopt glob2


In [1]:
import pandas as pd
import gensim.corpora as corpora
from gensim.models.ldamodel import LdaModel
from pprint import pprint
import matplotlib.pyplot as plt
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import termite 
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
def get_corpus(data, min_len=3):
    # Create a Dictionary: a mapping between words and their integer IDs
    id2word = corpora.Dictionary(data)
    
    # Remove tokens of 1 or 2 letters
    del_ids = [k for k,v in id2word.items() if len(v)<min_len]
    id2word.filter_tokens(bad_ids=del_ids)
    
    # Create a corpus: a list of documents represented as a BoW
    corpus = [id2word.doc2bow(text) for text in data]
    
    return id2word, corpus

In [3]:
def get_model(corpus, id2word, num_topics=3, passes=10, decay=0.5, iterations=50):
    coh_scores = []
    lda_model = LdaModel(
        corpus=corpus, 
        id2word=id2word, 
        num_topics=num_topics, 
        distributed=False,
        passes=passes, 
        update_every=1,
        alpha='auto', 
        eta=None, 
        decay=decay,
        eval_every=5,
        iterations=iterations, 
        per_word_topics=True)
    
    coherence_model_lda = CoherenceModel(
        model=lda_model, 
        texts=data, 
        dictionary=id2word, 
        coherence='c_v')
        
    coherence_lda = coherence_model_lda.get_coherence()
    print(f"Coherence score: {coherence_lda}")

    return lda_model, coherence_lda

def plot_coh_score(coh_scores, title, language, save=True): 
    fig, ax = plt.subplots(1, 1)
    ax.plot(range(2, 11), coh_scores, marker='o', linestyle='--')
    ax.title.set_text(title)
    ax.set_ylabel("Coherence score")
    ax.set_xlabel('Number of topics')
    ax.grid(True)
    if save:
        ax.get_figure().savefig("figures/LDA_coh_"+language, bbox_inches="tight")


In [4]:
def get_best_model(corpus, id2word, title, language, plot=False, save_plot=False):
    coh_scores = []
    for num_topics in range(2, 2):
    # for passes in range(10, 100, 10): 
    # for iterations in range(50, 100, 10): 
    # for decay in [0.6, 0.7, 0.8, 0.9, 1]:
        lda_model, coherence_lda = get_model(corpus, 
                                             id2word, 
                                             num_topics=num_topics, 
                                             passes=passes, 
                                             decay=decay, 
                                             iterations=iterations)
    coh_scores.append(coherence_lda)
    if coherence_lda == max(coh_scores):
        best_model = lda_model

    if plot:
        plot_coh_score(coh_scores, title, language, save_plot)

    return best_model 

<hr>

**German tasks**

<hr>

**English tasks**

In [7]:
folder = "gen_files/EN/"
# df = pd.read_csv(f"{folder}preprocessed/all_preprocessed_open_tasks_EN.csv")
df = pd.read_csv(f"{folder}preprocessed/trimmed_open_tasks.csv")

df_taskaspects = pd.read_csv(f"{folder}all_taskAspects_EN.csv")

# Keeping only the tasks that have one or more aspects of type CONCEPT
df = pd.merge(df, df_taskaspects, on="taskId", how="inner") 
df.reset_index(drop=True, inplace=True)

In [8]:
len(df.taskId.unique()), len(df_taskaspects.taskId.unique())

(951, 5706)

In [9]:
_df = df[["taskId", "description", "topic_id"]].drop_duplicates("taskId")
_df = _df.dropna(subset=["description"]).reset_index()
data = _df["description"].str.split().to_list() 

In [10]:
len(data)

951

In [11]:
id2word, corpus = get_corpus(data)

num_topics = 2 
passes = 20 
decay = 0.9
iterations = 100
lda_model_en, coherence_lda_en = get_model(corpus=corpus,
                                         id2word=id2word,
                                         num_topics=num_topics,
                                         passes=passes,
                                         decay=decay,
                                         iterations=iterations)

Coherence score: 0.28535525729911226


In [28]:
lda_model_en.get_topic_terms(0)

[(147, 0.107099764),
 (2, 0.07448792),
 (144, 0.05722191),
 (87, 0.01558184),
 (272, 0.010888486),
 (413, 0.010176769),
 (709, 0.009626901),
 (28, 0.007249978),
 (203, 0.0062585613),
 (242, 0.0051934486)]

In [16]:
# Visualization 
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model_en, corpus, id2word) 
vis.

PreparedData(topic_coordinates=             x    y  topics  cluster       Freq
topic                                          
0      0.14183  0.0       1        1  58.230459
1     -0.14183  0.0       2        1  41.769541, topic_info=         Term        Freq       Total Category  logprob  loglift
147  translat  562.000000  562.000000  Default  30.0000  30.0000
9       write  197.000000  197.000000  Default  29.0000  29.0000
2     sentenc  403.000000  403.000000  Default  28.0000  28.0000
149      hear  146.000000  146.000000  Default  27.0000  27.0000
480      ship   59.000000   59.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
455    forgot   15.873036   21.141665   Topic2  -5.4418   0.5864
8        word   15.073422   20.307316   Topic2  -5.4935   0.5750
242      find   20.113746   46.644781   Topic2  -5.2050   0.0318
52     answer   16.863990   29.078909   Topic2  -5.3813   0.3282
2     sentenc   23.470974  403.996832   Topic2  -5.0507  -1.9726

[130 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
52        1  0.412670     answer
52        2  0.584616     answer
248       1  0.940772       appl
248       2  0.062718       appl
1091      1  0.085605      april
...     ...       ...        ...
350       2  0.701124      world
9         1  0.055573      write
9         2  0.944735      write
905       1  0.092147  yesterday
905       2  0.921471  yesterday

[182 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])

In [21]:
documents = _df["description"].to_list()

# Infer topic distributions for each document
topic_distributions = lda_model_en.get_document_topics(corpus)

doc_to_topic = {}
for (i, d) in enumerate(topic_distributions): 
    doc_to_topic[i] = {u:v for (u,v) in d} 
    
df1 = pd.DataFrame.from_dict(doc_to_topic, orient='index').sort_index()
# Replace values that are less than 1/3 by NaN 
df1 = df1.mask(df1 < 1/3).reset_index()

In [22]:
df_tasks_topics = pd.concat([_df[["taskId"]], df1], axis=1) 
print(len(_df), len(df1))

951 951


In [23]:
df_tasks_topics

taskId  index         0         1
0    14ambh1obhw7TYMQE8lcC1      0       NaN  0.886949
1    25RGLvb2p0G5zulfX9xQOj      1  0.342221  0.657779
2    18Ccvc8NMJT5xqLv9nAgTH      2  0.647454  0.352546
3    3Jr6T26XL13aKRh31JX0xi      3       NaN  0.847030
4    3gbjpjewKN1aa5y4aN20Yw      4  0.466624  0.533376
..                      ...    ...       ...       ...
946  1vmTwnxXvLHaBpLOllxi8E    946       NaN  0.859428
947  aif5faqXBMr5BWFu35fcRC    947  0.974271       NaN
948  2lqfJ6gZiLr7w2cuDlFWvF    948       NaN  0.817819
949  16MreBTRQqA9TQI3zQU33G    949       NaN  0.777312
950  8gooLcJt0bz7yVHSaWd0MM    950       NaN  0.761384

[951 rows x 4 columns]

In [24]:
df_taskaspects.head()

taskId  aspectId
0  14ambh1obhw7TYMQE8lcC1_BT      9639
1  25RGLvb2p0G5zulfX9xQOj_BT      9639
2  18Ccvc8NMJT5xqLv9nAgTH_BT      9937
3  3Jr6T26XL13aKRh31JX0xi_BT      9633
4  3gbjpjewKN1aa5y4aN20Yw_BT     11401

In [31]:
df_task_to_aspects = df_taskaspects.groupby(by="taskId")["aspectId"].apply(list).reset_index()
df_task_to_aspects.head()

taskId                                        aspectId
0     12QfXkR96hp8Hyu8mSN1gm  [183, 184, 185, 186, 187, 17042, 18153, 19487]
1  12QfXkR96hp8Hyu8mSN1gm_BT  [183, 184, 185, 186, 187, 17042, 18153, 19487]
2  12QfXkR96hp8Hyu8mSN1gm_RD  [183, 184, 185, 186, 187, 17042, 18153, 19487]
3  12QfXkR96hp8Hyu8mSN1gm_RI  [183, 184, 185, 186, 187, 17042, 18153, 19487]
4  12QfXkR96hp8Hyu8mSN1gm_RS  [183, 184, 185, 186, 187, 17042, 18153, 19487]

In [32]:
len(df_tasks_topics), len(df_tasks_topics.taskId.unique()), len(df_task_to_aspects)

(951, 951, 5706)

In [33]:
df_task_topic_aspect = pd.merge(df_tasks_topics, 
                                df_task_to_aspects, 
                                on="taskId", 
                                how="inner").drop(columns=["index"])
df_task_topic_aspect.sample(5)

taskId         0         1  \
439  7VTCPEeBUKD6bYzU2It4PD  0.926826       NaN   
941  68RE5Cw3jm48eggO9ZPAJf  0.924533       NaN   
56   aS38m2r9A37aEFQXHdmakv  0.925564       NaN   
216  aW1X8eF8Xc87NArOOacU1Z  0.939702       NaN   
264  9p3ySDQHqFg6c3SqwKxw8Q       NaN  0.788442   

                                              aspectId  
439  [1415, 1631, 1417, 1416, 1418, 1419, 1632, 158...  
941                           [349668, 381286, 381288]  
56      [183, 184, 185, 186, 187, 17042, 18153, 19487]  
216     [183, 184, 185, 186, 187, 17042, 18153, 19487]  
264  [1415, 1631, 1417, 1416, 1418, 1419, 1632, 159...

In [34]:
df_task_topic_aspect[0]

0           NaN
1      0.342221
2      0.647454
3           NaN
4      0.466624
         ...   
946         NaN
947    0.974271
948         NaN
949         NaN
950         NaN
Name: 0, Length: 951, dtype: float64

In [36]:
df_topic_0 = df_task_topic_aspect[[0, "aspectId"]].dropna(subset=[0]).reset_index(drop=True)
df_topic_1 = df_task_topic_aspect[[1, "aspectId"]].dropna(subset=[1]).reset_index(drop=True)
# df_topic_2 = df_task_topic_aspect[[2, "aspectId"]].dropna(subset=[2]).reset_index(drop=True)

In [38]:
from collections import Counter
import itertools
aspects_0 = df_topic_0["aspectId"].to_list()
aspects_0 = list(itertools.chain.from_iterable(aspects_0))
occurrences_0 = Counter(aspects_0)

aspects_1 = df_topic_1["aspectId"].to_list()
aspects_1 = list(itertools.chain.from_iterable(aspects_1))
occurrences_1 = Counter(aspects_1)

# aspects_2 = df_topic_2["aspectId"].to_list()
# aspects_2 = list(itertools.chain.from_iterable(aspects_2))
# occurrences_2 = Counter(aspects_2)

In [39]:
# define threshold t as a function of the values of the occurrences.
# defining t as a fixed value risks of having an empty dict result. 
from statistics import median 
t = median(occurrences_0.values())
retained_occ = dict(filter(lambda x: x[1] > t, occurrences_0.items()))
retained_aspects_0 = list(retained_occ.keys())
print(len(retained_aspects_0))

t = median(occurrences_1.values())
retained_occ = dict(filter(lambda x: x[1] > t, occurrences_1.items()))
retained_aspects_1 = list(retained_occ.keys())
print(len(retained_aspects_1))

# t = median(occurrences_2.values())
# retained_occ = dict(filter(lambda x: x[1] > t, occurrences_2.items()))
# retained_aspects_2 = list(retained_occ.keys())
# print(len(retained_aspects_2))

107
87


In [46]:
for i in range(3,4): 
    print(i)

3


In [40]:
from itertools import combinations
from scipy.spatial.distance import pdist, squareform
import numpy as np

def jaccard_dissimilarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return  1 - intersection / union 


def average_pairwise_jaccard(sets):
    dissimilarities = [jaccard_dissimilarity(s1, s2) for s1, s2 in combinations(sets, 2)]
    return np.mean(dissimilarities)

# Example usage
sets = [set(retained_aspects_0), set(retained_aspects_1)]
avg_jaccard = average_pairwise_jaccard(sets)
print(f'Average pairwise Jaccard dissimilarity: {avg_jaccard}')


Average pairwise Jaccard dissimilarity: 0.484375


In [41]:
# Verify the number of common elements between the aspects of each topic 
# to assess their disparity. 
print(len(set(retained_aspects_0).intersection(set(retained_aspects_1))))
# print(len(set(retained_aspects_2).intersection(set(retained_aspects_1))))
# print(len(set(retained_aspects_0).intersection(set(retained_aspects_2))))

66


subtopics

In [55]:
tasks_topic0 = df_tasks_topics[["taskId", 0]].dropna(subset=[0]).reset_index(drop=True)
tasks_topic0 = pd.merge(tasks_topic0, _df, on="taskId", how="inner").drop(columns=[0,"index"])

data = tasks_topic0["description"].str.split().to_list() 
id2word, corpus = get_corpus(data)

num_topics = 3 
passes = 20 
decay = 0.9
iterations = 100
lda_model_en, coherence_lda_en = get_model(corpus=corpus,
                                         id2word=id2word,
                                         num_topics=num_topics,
                                         passes=passes,
                                         decay=decay,
                                         iterations=iterations)

Coherence score: 0.3734310209940581


In [56]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model_en, corpus, id2word) 
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.233905 -0.007442       1        1  48.080714
1     -0.133680 -0.074329       2        1  33.996273
0     -0.100225  0.081771       3        1  17.923013, topic_info=       Term        Freq       Total Category  logprob  loglift
518    gwen  519.000000  519.000000  Default  30.0000  30.0000
494    dora  669.000000  669.000000  Default  29.0000  29.0000
553   phone  298.000000  298.000000  Default  28.0000  28.0000
476    chef  278.000000  278.000000  Default  27.0000  27.0000
711  robert  208.000000  208.000000  Default  26.0000  26.0000
..      ...         ...         ...      ...      ...      ...
151   three   63.011163  244.214528   Topic3  -5.0381   0.3643
318  realli   65.542662  292.042925   Topic3  -4.9987   0.2249
343     ask   64.048098  355.692871   Topic3  -5.0218   0.0047
107    take   54.854581  322.777917   Topic3  -5.1767  -0.0532
84     name   53.643765  183.901577   Topic3  -5.1991   0.4870

[253 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
939       1  0.975297   abbi
939       2  0.009289   abbi
939       3  0.013933   abbi
852       1  0.972408  activ
852       2  0.010345  activ
...     ...       ...    ...
263       2  0.237358   year
263       3  0.380877   year
666       1  0.016918   yuck
666       2  0.913579   yuck
666       3  0.084591   yuck

[518 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

**English Aspects**

In [ ]:
d = pd.read_csv("data/taskAspects_EN.csv")
len(d.aspectId.unique())

In [ ]:
i = pd.read_csv("data/preprocessed_concept_aspects_EN.csv")
i

In [ ]:
df = pd.read_csv("data/preprocessed_concept_aspects_EN.csv")
df.dropna(subset=["description"], inplace=True)
data = df["description"].str.split().to_list() 
title = "Coherence score by number of topics in english tasks" 

id2word, corpus = get_corpus(data)

num_topics = 3 
passes = 20 
decay = 0.9
iterations = 100
lda_model_aspects, coherence_lda_aspects = get_model(corpus=corpus,
                                                     id2word=id2word,
                                                     title=title,
                                                     num_topics=num_topics,
                                                     passes=passes,
                                                     decay=decay,
                                                     iterations=iterations)

In [ ]:
lda_model_aspects.print_topics() 

In [ ]:
# Visualization 
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model_aspects, corpus, id2word) 
vis

<hr>

Functions that come with gensim LDAModel: <br>
- get_document_topics(bow[, ...]) 	Return topic distribution for the given document bow, as a list of (topic_id, topic_probability) 2-tuples.<br>
- get_term_topics(word_id[, minimum_probability]) 	Returns most likely topics for a particular word in vocab.<br>
- get_topic_terms(topicid[, topn]) 	Return a list of (word_id, probability) 2-tuples for the most probable words in topic topicid. <br>
- show_topic(topicid[, topn]) Return a list of (word, probability) 2-tuples for the most probable words in topic topicid.<br>
- top_topics(corpus[, num_words]) 	Calculate the Umass topic coherence for each topic.<br>
  https://tedboy.github.io/nlps/generated/generated/gensim.models.LdaModel.html                                             